# BASIN moveout plot notebooks
This notebook makes moveout plots of the stacked cross correlations between broadbands and nodal stations. It is highly inspired by Julian Schmitt's work, modified by Marine Denolle (mdenolle@uw.edu)

In [3]:
# Packages - set GR as plotting backend 
#include("/Users/julianschmitt/Documents/Schoolwork/Seismology/SeisCore.jl/src/SeisCore.jl")
using Pkg, SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, Distributed, JLD2, Statistics, PyCall, Glob, StructArrays, ColorSchemes, Plots.PlotUtils, HDF5, AbstractFFTs
include("../../src/process.jl")
include("../src/utils.jl")
# ENV["GR"] = ""
# Pkg.build("GR")
loadcolorscheme(:cm_maxamp,ColorSchemes.gist_heat.colors[end-50:-1:1], "maxamp color", "for waveform plot");

In [4]:
# metaparams for plots
n_derivatives = 2
n_deriv_names = ["Raw","PGD","PDV"]
#plot_sorttype = "Latitude" # other options include "Longitude" and (soon) "Distance" (in progress...) 
years = [2017]#, 2018, 2019]
components = ["ZZ","EE","EN","EZ","NE", "NN", "NZ","ZE","ZN", "ZZ"]
source_stations = ["SVD"]#["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]
rootdir = "/Users/marinedenolle/BASIN" # chose root directory for plots
lw, scale, frequency_plots = 0.5, 3, [[0.1,0.2],[0.15, 0.25],[0.2,0.3],[0.1,0.35]]
locations = DataFrame(CSV.File("/Users/marinedenolle/BASIN/C4-Project.jl/docs/modified_nodal.csv"));

In [13]:
files = glob("nodestack/*",rootdir);
println(files)
filter!(x -> occursin("SVD", x), files)


["/Users/marinedenolle/BASIN/nodestack/2017", "/Users/marinedenolle/BASIN/nodestack/2018", "/Users/marinedenolle/BASIN/nodestack/2019", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017.h5", "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5"]


2-element Vector{String}:
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017.h5"
 "/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5"

In [15]:
files[2]

"/Users/marinedenolle/BASIN/nodestack/CI.SVD_2017_allday.h5"

## function to plot per distance range

In [48]:
function plot_by_latitude(corrs::Array{CorrData,1}, freqs::Array{Array{Float64,1},1}, 
                            attr::Array{String,1}=["","","",""], tname::String=""; src_geo::Any=nothing, fs::Float64=20.)
    source = attr[1]
    println(source)
    println("Sorting by distance")
    println(tname)
    plot_sorttype == "Latitude"
    println(corrs[1].loc.lat)
    
    sort_method = sort([corr.loc.lat for corr in corrs])
#         sort_method = sort([corr.latitude for corr in corrs])#sort([get_dist(source_geo, corr.loc) for corr in corrs])
    p = sortperm([corr.loc.lat for corr in corrs])#[get_dist(source_geo, corr.loc) for corr in corrs])
        #println("Not currently a supported stacktype")


    lmin, lmax = minimum(sort_method), maximum(sort_method)
    new_corrs, n_corrs = corrs[p], length(corrs)
    y_labels = (round.(collect(range(0, 10*n_corrs, length=10)),digits=3),round.(collect(range(lmin, lmax, length=10)),digits=3))
    for freq_pair in freqs
        NS_plot = plot(xlims = (-60,60), ylims = (-10,n_corrs*10+15), 
                    yticks=y_labels,xlabel = "Time (s)", ylabel = "Station Nodes by $plot_sorttype (lat)",  
                    xtickfontsize=5,ytickfontsize=5,fontsize=5, xguidefontsize = 10, yguidefontsize = 10,
                    legendfontsize = 10, title="$(attr[3]) $(attr[2]) from $(attr[1]): $(freq_pair[1]) to $(freq_pair[2])",
                    titlefontsize= 10)
        # bandpass correlations 
        corrs_processed = []
        max_amp = []
        ys, maxloc = [], []
        for (ind, corr) in enumerate(new_corrs)
            try
                # bandpass, scale, shift, and add to plot
                dist = corr.dist #ind*5#get_dist(source_geo, corr.loc)
                taper!(corr)
                processed = bandpass(corr.corr, freq_pair[1], freq_pair[2], 20.)
                nwin2 = convert(Int64, round(length(processed)/2))
                t2=convert(Int64, floor(nwin2-(dist/5)*corr.fs))
                t1=convert(Int64, ceil(nwin2-(dist/1.5)*corr.fs))
                trange = collect(t1:1:t2) # anticausal window of ballistic waves
                imax = findmax(processed[trange])[2]+t1
                push!(maxloc, imax)
                push!(ys, ind*5)
                push!(max_amp, findmax(processed[trange]))
                push!(corrs_processed, processed./findmax(processed)[1].*10 )
            catch e
                println(e)
            end
        end
        println("done with the loop")
        # get the max amp
        mmax = findmax(max_amp)[1][1]
        T = collect(-300.:1/fs:300.)
#         crap=reshape(corrs_processed,(length(corrs_processed),length(corrs_processed[1])))
        crap=[]
        for i in collect(1:length(corrs_processed))
#             println(i,"",mmax)
#             println(i,"",max_amp[i][1],"",mmax)
            push!(crap,corrs_processed[i]./max_amp[i][1].*mmax .+ 10*i)

        end
        
#         println(size(transpose(crap)))
#         println(length(max_amp))
#         println(length(mmax))
#         println(crap)
        plot!(NS_plot, T, crap, color=:cm_maxamp, colorbar_title="Normalized Maximum Amplitude", 
                line_z=getindex.(max_amp,1)', fmt = :png, linewidth = 0.5, reuse = false, legend = false)    
        plot!(size=(350,400),dpi=500)
        
        # add dots where max_amp is chosen 
        dot_locations = [amp[2] for amp in max_amp]
        y = 10 .* collect(0:length(max_amp))
#         scatter!(getindex.(getindex.(max_amp,2),1) , y, color="black", markersize=2)
        mloc = [T[ind] for ind in maxloc]
        scatter!(mloc, ys, color="black", markersize=1.2)
        filepath = joinpath("/Users/marinedenolle/BASIN/nodestack_windowed/$(attr[1])/")
#         println(filepath)
        #         # ensure filepath is valid 
        DIR = dirname(filepath)
        if !isdir(DIR)
            mkpath(DIR)
        end
        print(filepath)
        println(tname)
        println(filepath*"waveforms_$(attr[2])_$(freq_pair[1])to$(freq_pair[2])_$(tname).png")
        png(NS_plot,filepath*"waveforms_$(attr[2])_$(freq_pair[1])to$(freq_pair[2])_$(tname).png")
    end
end

plot_by_latitude (generic function with 8 methods)

## Stackplot Script for BASIN Data

In [50]:
plot_sorttype = "Latitude"
for f in files
    year = split(split(split(f,"/")[end],"_")[end-1],".")[1]
# 
    try  
        file = h5open(f,"r")
        for comp in components
            for (ind, sktype) in enumerate(["linear"])#,"pws","robust"])
                for filt in ["NO.G1"]#,"NO.B2","NO.B3", "NO.B4","NO.B5","NO.B6","NO.G1","NO.G2","NO.G3","NO.G4"]
                    source, stacktype, component, filter = convert(String, split(f, ".")[end-1][1:3]), sktype, comp, filt
                    fcorrs = get_corrs(file, stacktype, component, filter)
                    if length(fcorrs) > 1
#                         println("Plotting $filt $comp $sktype for $source in $year")
                        attributes = [source,component,stacktype]
                        println("what the hell with this string")
                        println(typeof("$(filt)_allday"))
                        plot_by_latitude(fcorrs, frequency_plots,attributes,"$(filt)_allday")
#                         println("finised plotting")
                    end
                end
            end
        end
    catch e
        println(e)
    end
end

ErrorException("Cannot access file /Users/marinedenolle/BASIN/nodestack/CI.SVD_2017.h5")
what the hell with this string
String
SVD
Sorting by distance
NO.G1_allday
34.15801
done with the loop
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/NO.G1_allday
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/waveforms_ZZ_0.1to0.2_NO.G1_allday.png
done with the loop
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/NO.G1_allday
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/waveforms_ZZ_0.15to0.25_NO.G1_allday.png
done with the loop
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/NO.G1_allday
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/waveforms_ZZ_0.2to0.3_NO.G1_allday.png
done with the loop
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/NO.G1_allday
/Users/marinedenolle/BASIN/nodestack_windowed/SVD/waveforms_ZZ_0.1to0.35_NO.G1_allday.png
what the hell with this string
String
SVD
Sorting by distance
NO.G1_allday
34.15801
done with the loop
/Users/marinedenolle/BASIN/nodestack_

HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 1:
  #000: H5Fdeprec.c line 156 in H5Fis_hdf5(): unable to determine if file is accessible as HDF5
    major: File accessibility
    minor: Not an HDF5 file
  #001: H5VLcallback.c line 3769 in H5VL_file_specific(): file specific failed
    major: Virtual Object Layer
    minor: Can't operate on object
  #002: H5VLcallback.c line 3699 in H5VL__file_specific(): file specific failed
    major: Virtual Object Layer
    minor: Can't operate on object
  #003: H5VLnative_file.c line 384 in H5VL__native_file_specific(): error in HDF5 file check
    major: File accessibility
    minor: Unable to initialize object
  #004: H5Fint.c line 1073 in H5F__is_hdf5(): unable to open file
    major: File accessibility
    minor: Unable to initialize object
  #005: H5FD.c line 723 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #006: H5FDsec2.c line 355 in H5FD__sec2_open(): unable to open file: name = '/Us